In [ ]:
import cv2
import numpy as np
from mtcnn.mtcnn import MTCNN

#Cargar elementos del filtro con canal alfa para el overlap 
chaplin_hat = cv2.imread("images/chaplin_hat.png", cv2.IMREAD_UNCHANGED)
chaplin_mustache = cv2.imread("images/chaplin_moustache.png", cv2.IMREAD_UNCHANGED)

print(chaplin_hat)
print(chaplin_mustache)
# Carga del modelo MTCN 
mtcnn_det = MTCNN()


# Webcam connection
video_capture = cv2.VideoCapture(0)

#funcion de overlay
def overlay(fondo, img,x,y):
    
    fondo_width = fondo.shape[1]
    fondo_height = fondo.shape[0]
    img_width = img.shape[1]
    img_height = img.shape[0]

    #bordes si se sale nno hacer nada
    if x >= fondo_width or y >= fondo_height:
        return fondo
    
    #limmites 
    min_height = min(img_height,fondo_height - y)
    min_width = min(img_width,fondo_width - x)

    #recorte de la imagen par que tenga el tamaño justo
    overlay = img[0:min_height, 0:min_width]

    #separar canales 
    chanels = overlay[:,:,:3]
    mask = overlay[:,:,3:] / 255.0

    fondo[y:y+min_height, x:x+min_width] = (1 - mask) * fondo[y:y+min_height, x:x+min_width] + mask * chanels
    
    return fondo


while True:
    ret, frame = video_capture.read()    
    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = mtcnn_det.detect_faces(rgb_frame)
    print(results)
    #Extracion de puntos de interes 
    for result in results:
        keypoints = result['keypoints']
        print(keypoints)
    
    #puntos de interes
    left_eye = keypoints['left_eye']
    right_eye = keypoints['right_eye']
    nose = keypoints['nose']
    mouth_left = keypoints['mouth_left']
    mouth_right = keypoints['mouth_right']

    #calcular posicion sombrero
    
    #calcular el centro de los ojos para determinar el centro de la cara 
    eye_center = (
        int((left_eye[0] + right_eye[0]) / 2),
        int((left_eye[1] + right_eye[1]) / 2)
    )
    
    # la diferencia entre el final y el principio es el ancho
    # el 2 es para cojer un poco mas de espacio y que no quede justo modificar si falla  
    hat_width = int(2*abs(right_eye[0] - left_eye[0]))

    #nuevo alto = nuevo ancho *(ancho original /alto original) para respetar las proporciones originales 
    hat_height = int(hat_width * (chaplin_hat.shape[0] / chaplin_hat.shape[1]))

    # empieza a dibujar en la esquina izquierda de la cabeza 
    x_hat = int(eye_center[0] - (hat_width/2))
    # empieza un poco por encima de el centro de los ojos el 0.8 hay que ajustarlo
    y_hat = int(eye_center[1] - (hat_height*0.8))

    #longitud del bigote ajustar factor
    mouth_center = (
        int((mouth_left[0] + mouth_right[0]) / 2),
        int((mouth_left[1] + mouth_right[1]) / 2)
    )


    mustache_width = int(2*abs(mouth_left[0] - mouth_right[0]))
    
    mustache_height = int(mustache_width * chaplin_mustache.shape[0] / chaplin_mustache.shape[1])

    #ajustar valor de y si necesario
    x_mustache = int(mouth_center[0] - mustache_width/2)
    y_mustache = int(mouth_center[1] - (mustache_height * 0.4))


    #dibujar en la imagenel objeto en los puntos
    chaplin_mustache_resize = cv2.resize(chaplin_mustache, (mustache_width, mustache_height))
    chaplin_hat_resize = cv2.resize(chaplin_hat, (hat_width,hat_height))

    frame = overlay(frame, chaplin_hat_resize,x_hat,y_hat)
    frame = overlay(frame, chaplin_mustache_resize,x_mustache,y_mustache)

    # Shgow results
    grey_filter = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cv2.imshow('Chaplin filter', grey_filter)
    
    # 'd' to change detector, Esc to finish
    key = cv2.waitKey(1) & 0xFF
    if key == 27:
        break

# Close windoews and release camera
video_capture.release()
cv2.destroyAllWindows()


[[[0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  ...
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 [[0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  ...
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 [[0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  ...
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 ...

 [[0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  ...
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 [[0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  ...
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 [[0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  ...
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]]
[[[0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  ...
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 [[0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  ...
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 [[0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  ...
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 ...

 [[0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  ...
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 [[0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  ...
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 [[0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  ...
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]]
[{'box': [246, 177, 158, 223

ValueError: operands could not be broadcast together with shapes (139,212,1) (0,212,3) 

: 